Current Questions: how do input image - flatten or include rows and columns?

In [2]:
# Autoreload imports of modified .py modules
import autoreload
%load_ext autoreload
%autoreload 2  

In [3]:
import os

import numpy as np

# NN Modules
import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils

# OpenCV
import cv2

# UNET module
import solar_panel_detection.unet as unet

Using TensorFlow backend.


In [4]:
def image_to_feature_vector(image, size=(32, 32)):
    '''
    Resize the image to a fixed size, then flatten the image into
    1D list of raw pixel intensities.
    Note: resizing will lose information about image
    Note: can choose different interpolation schemes
    
    Parameters
    ----------
    image: image-like
        input image
    size: tuple
        resolution of output vector
    
    Returns
    -------
    resized image
    '''
    return cv2.resize(image, size).flatten()

In [34]:
# Read in example photo
imTrain = cv2.imread('data/3385828/11ska460890.tif')

# Load dummy target image as grayscale
imTarget = cv2.imread('data/11ska460890_dummy.tif', 0)
# Replace all non-black pixel RGB (0, 0, 0) with 255 (white) for target
imTarget[imTarget>0] = 255
imTarget[imTarget==0] = 1
imTarget[imTarget==255] = 0
imTarget = imTarget*255

# Load dummy image again and make binary target for training
target = cv2.imread('data/11ska460890_dummy.tif', 0)
target[target>0] = 255
target[target==0] = 1
target[target==255] = 0

In [35]:
imTarget

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [36]:
# Resize images
resize = (128, 128)
imTrain = cv2.resize(imTrain, resize)  
imTarget = cv2.resize(imTarget, resize)  
target = cv2.resize(target, resize)

In [37]:
# View images
cv2.imshow('image', imTarget)
cv2.waitKey(0)

-1

In [38]:
# Normalize and scale training data
training = imTrain.astype('float32')
trainMean = np.mean(training)
trainStd = np.std(training)
training -= trainMean
training /= trainStd

In [39]:
# Convert target to categorical variables for cross-entropy metric
nClasses = 2
target = to_categorical(target, num_classes=nClasses)

In [43]:
# Reshape training and target data with n_images
nImages = 1
training = training.reshape(nImages,resize[0],resize[1],3)
target = target.reshape(nImages,resize[0],resize[1],nClasses)

In [44]:
# Create UNET object and create model
UNET = unet.Unet(resize[0], resize[1], 3)
modelUnet = UNET.create_unet()

In [46]:
# Fit model
modelUnet.fit(x=training, y=target, epochs=3, batch_size=None, verbose=2)

Epoch 1/3
 - 4s - loss: 0.7245 - categorical_accuracy: 0.0103
Epoch 2/3
 - 2s - loss: 0.5968 - categorical_accuracy: 0.9998
Epoch 3/3
 - 2s - loss: 0.0427 - categorical_accuracy: 0.9998


In [52]:
# Get predictions and output image of predictions
predictions = modelUnet.predict(x=training, batch_size=None)
predictions[predictions>0] = 255
predImage = np.argmax(predictions[0], axis=2)
cv2.imwrite('predImage.jpg', predImage)

In [83]:
cv2.imshow('Prediction', predImage)
cv2.waitkey(0)

AttributeError: module 'cv2.cv2' has no attribute 'waitkey'